# 概要

このノートでは、DIGITSとCaffeを使用して航空画像から物体を検出する方法について、いくつかの例を上げて説明していきます。個々の例は、NOAA（アメリカ海洋大気庁）が実施した、海を撮影した航空画像からクジラの個体を判別するコンテスト「Right Whale Recognition(https://www.kaggle.com/c/noaa-right-whale-recognition) 」を参考にしています。

図1に母鯨と子鯨を含む画像例を示します。

![Right whale example](right_whale_example.png)
<h4 align="center">図1: 鯨の親子</h4> 

ここでは少し違った問題に取り組みたいと思います。クジラかどうかを判別するのではなく、畳み込みニューラル・ネットワーク（CNN）をトレーニング(学習)することで画像内のクジラの位置を特定していきます。画像の中から目的の物体が存在する領域を見つけ出すことを「物体検出」と呼ぶことがあります。前述のコンテストの多くの入賞者が、画像内でクジラが存在する領域を検出および特定してから、切り出して正規化した画像を使用してクジラを判別することで、スコアが高くなったと言っています。

## 物体検出方法1:  スライディング・ウィンドウ

畳み込みニューラル・ネットワーク（CNN）を使用して画像の中から目的の物体が存在する領域を見つけ出す方法はいくつかあります。最もシンプルな方法は、目的の物体と物体以外を区別する画像パッチについてCNN分類器をトレーニング(学習)することです。図2は、クジラのパッチと背景のパッチを識別するCNNのアーキテクチャを示しています。

![patch based model](patch_based.png)
<h4 align="center">図2: CNN画像分類のパッチベース学習</h4> 

このような分類器を備えることで、大きな画像データを構成する小さなパッチを個々に調べ、場合によっては重なり合うパッチを使用して、クジラの存在を確認できるようになります。それでは実行していきましょう。
このノートでは2つのデータセットを使用できます。最初のデータセットは、クジラを含む広域の海の画像で構成されています。このデータセットはdata_336x224にあります。2番目のデータセットは、切り出した最大4500枚のクジラの顔画像と、同じ画像からランダムに切り出した4500枚の追加画像で構成されています。このデータセットはそれぞれdata/train/facesおよびdata/train/backgroundにあります。この2番目のデータセットを使用してDIGITSの分類器をトレーニング(学習)します。


<a id='question1'></a>
### 質問1

ランダムに切り出した画像をバックグラウンド・セットとして使用することで、どのような問題が発生する可能性があるでしょうか？

答え: [ここ](#答え1)をクリック

[ここ](/digits/)をクリックしてDIGITSを開きます。

最初にデータセットをDIGITSにインポートする必要があります。[Datasets]の[Images]ドロップダウンリストから[classification]データセットを選択します。[New Image Classification Dataset]パネルが表示されたら、次の前処理オプションを設定します。

![whale faces import](whale_faces_import.png)

これにより、face/notfaceデータセットが256x256のカラー画像としてインポートされ、25%が検証(バリデーション)データセットとして分けられます。DIGITSは、画像のフォルダ構造から2つのクラスの名前を自動的に識別します。 

インポート処理には数分かかります。インポートが完了したら、DIGITSのメイン画面に戻ってwhale_facesデータセットを再度入力し、[Explore the db]ボタンをクリックすると、各クラスの画像サンプルを確認できます。次のような画像が表示されます。

![whale face examples](whale_face_examples.png)

ここで、2つのシンプルなクラスを使用して、このデータセットについてCNN分類器をトレーニング(学習)します。DIGITSのメイン画面に戻り、[Models]の[Images]ドロップダウンリストから[classification]モデルを選択します。表示される[New Image Classification Model]パネルで、ほとんどのオプションは初期設定のままにします。
次のみカスタマイズが必要になります。

* 作成したwhale_facesデータセットを選択します 
* [Standard Networks]で[Alexnet]を選択します
* トレーニング・エポック数を5に設定します
* 「whale_faces_baseline」という名前のモデルを選択します。

このパネルは次のようになります。

![DIGITS New Image Classification Model panel](whale_faces_digits_model.png)

[Create] をクリックしてモデルのトレーニング(学習)を開始します。

モデルのトレーニング・ロスおよび検証セットのロスと精度を示すグラフがリアルタイムで表示されます。トレーニング(学習)が進むにつれてロスが低下し、精度が上昇するはずです。トレーニング(学習)が完了するまで数分かかります。最終的には検証セットの精度が約98%となり、非常に高精度なクジラの顔画像分類器が作成されます。

画像URLの入力フィールドに「`/home/ubuntu/data/whale/data/train/face/w_2606.jpg`」と入力して[Classify One]をクリックすることにより、個々の画像パッチに対して分類器をテストできます。[Show visualizations and statistics]チェックボックスをオンにして、分類を行うためにCNNが画像内のどのような物に反応しているかを確認してください。
以上で、スライディング・ウィンドウを使って広域の航空画像からクジラの顔を検出するためにこのノートで使用するモデルが作成されました。トレーニング(学習)したモデルのDIGITSでのジョブ番号は、そのモデルのページのURLの末尾にあります。
たとえば、URLが「`localhost/models/20160525-014512-ce40`」である場合、ジョブ番号は「`20160525-014512-ce40`」になります。

この番号は、次のコードでJOB_NUMとして参照する場合に必要になります。
次のコードを、使用するモデルの正しいJOB_NUMに変更して実行します。実行には約30秒かかります。モデルに入力すると、重なり合わない256x256の各グリッド枠の予測クラスを示す配列とともに、ランダムに選択された広域のテスト画像が出力されます。 

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import time

JOB_NUM = '20160725-162841-9732'  ## Remember to set this to be the job number for your model

MODEL_FILE = '/home/ubuntu/digits/digits/jobs/' + JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/home/ubuntu/digits/digits/jobs/' + JOB_NUM + '/snapshot_iter_270.caffemodel'    # Do not change

# Choose a random image to test against
RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = 'data/samples/w_' + RANDOM_IMAGE + '.jpg'                   

# Tell Caffe to use the GPU
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

前述のコードを複数回実行することで、一部の例では、このベースライン・モデルとスライディング・ウィンドウによる物体検出方法により、クジラの顔の位置を高い確率で特定し、より多くのクジラを検出できることがわかります。しかし同時に、海面の波や日光の反射によってモデルの精度が簡単に低下してしまうこともわかると思います。

<a id='question2'></a>
### 質問2

このモデルの分類の精度を向上させるにはどのような方法がありますか？

答え: [ここ](#答え2)をクリック

重なり合わないグリッド枠を使用したスライディング・ウィンドウによる位置検出では、一部のグリッド枠にクジラの顔が部分的にしか含まれず、これが分類エラーを引き起こす原因となる可能性があります。残念なことに、スライディング・ウィンドウ方式では、グリッド枠の重なりを増やすと計算時間が急激に上昇します。さらに、重なりによる分類を最終的な分類「ヒートマップ」に組み合わせる方法について決定する必要もあります。このための一般的な方法がNMS（Non-Maximum Suppression）アルゴリズムです。

<a id='question3'></a>
### 質問3

グリッド枠を重ね合わせるのに必要な計算時間の増加にはどのように対処すればよいでしょうか？

答え: [ここ](#答え3)をクリック

### オプション演習にチャレンジしてみよう

1. グリッド枠のサイズを256x256のまま維持しながら、グリッド枠間の重なりを増やし、より精細な分類マップを得られるようコードを修正します。

2. 予測を行うためにネットワークに渡す複数のグリッド枠をまとめてバッチ処理するようコードを修正します。

先ほど紹介したように、このスライディング・ウィンドウによる物体検出方法の利点は、（より広範に入手可能な）パッチ・ベースのトレーニング・データのみを使用して検出器のトレーニング(学習)を行えることです。ただし、以下のような欠点もあります。

* 特にグリッド枠間の重なりが大きい場合、予測に時間がかかります。
* 重なりが大きいと、冗長な計算が大量に生じて誤検出に強いバランスの取れたトレーニング・データセットを生成するのが困難になります。
* 物体検出のスケール不変性を実現するのが難しくなります

## 物体検出方法 2: 候補の生成と分類

実際にはこの2つ目の方法のデモは行いませんが、完全を期すためにここで説明します。
スライディング・ウィンドウ形式の分類CNNを使用する代わりに、計算コストが低く精度が高いが誤検出が発生しやすいアルゴリズムを使用して、検出候補を生成することもできます。このプロセスに使用されるアルゴリズムには、たとえば、カスケード型分類器や選択検索などがあります。これらの検出候補は次にCNNに渡され、物体の種類別に分類されるか、バックグラウンド・ノイズとして除去されます。

これらの候補生成アルゴリズムでは、通常、スライディング・ウィンドウによる物体検出方法でテストが必要なグリッド枠と比べ、きわめて少ない数の、CNNで分類される画像パッチが生成されます。さらに、これらの検出候補は、並列処理の恩恵を受けるために、CNNに入力する前にまとめてバッチ処理することができます。

図3は車両検出にどのようにこのアプローチが使われているかを示しています。

![candidate generation example](candidate_generation_example.png)
<h4 align="center">図3: 候補領域生成の前処理とCNNによる分類</h4> 

この方法には以下のような利点があります。

* テストする検出候補数の削減による高速化
* 候補生成アルゴリズムによっては、物体の位置特定精度を向上可能

この方法には以下のような欠点があります。

* より複雑な多段処理パイプライン
* 候補を生成するための追加モデルの構築やトレーニング
* 無視できない誤検出率
* 生成される候補の数によって左右される可変的な推論時間

## 物体検出方法3: 全畳み込みネットワーク (FCN)

前述のとおり、ウィンドウが重なるスライディング・ウィンドウ方式では、大量の冗長な計算が行われます。ありがたいことに、この冗長は巧みな工夫で避けることができます。一般的に使用される、AlexnetのようなCNNの分類目標に向けた全結合層は、ほとんど労力を要さずに畳み込み層に置き換えることができます。これらの置き換え層には前の層に対して出力された特徴マップと同じサイズの畳み込みフィルタがあり、フィルタ数は置き換えられる全結合層のニューロン数と同じです。この置き換えを行う利点は、分類するためにさまざまなサイズの画像をネットワークに入力できる点です。入力画像がネットワークの予想画像サイズ（これをネットワークの受容野と呼ぶ）より小さい場合も、その画像に対して単一の分類を得ることができます。
ただし、画像が受容野より大きい場合は、スライディング・ウィンドウ方式で取得するものによく似た、分類のヒートマップが得られます。

クジラ検出問題でこれを試してみましょう。

DIGITSに戻り、モデル画面の右上にある[Clone]ボタンを使用して、クジラの顔検出ベースライン・モデルのクローンを作成します。次に、Alexnetネットワーク・アーキテクチャを選択すると同時に[Customize]オプションを選択します。


<a id='exercise1'></a>
### 演習:

全結合（InnerProduct）層fc6を6x6畳み込みフィルタが256個ある畳み込み層に置き換え、全結合層fc7を完全に削除し、fc8を1x1畳み込みフィルタが2個ある畳み込み層に置き換えます。このモデルをうまくトレーニング(学習)できるかどうか確認します。

注：これは、オリジナルのAlexNetにおける層の直接的な置き換えとは異なります。実際のところ、トレーニング(学習)可能なパラメータ数を削減しています。これは、ネットワークの完全畳み込みバージョンのトレーニング(学習)にはより多くのGPUメモリが必要で、これらのクラウド・ベースのマシンではメモリを使い果たしてしまうからです。

答え: [ここ](#答え4)をクリック

必要な変更を加えた後は、モデルのトレーニング(学習)を行い、最高98%のほぼ同等の検証精度を達成する必要があります。

これで、このモデルを使用して、広域の航空画像全体の分類ヒートマップを直接計算できるようになりました。事実上は、依然としてスライディング・ウィンドウ分類を行っていますが、すべてのスライディング・ウィンドウ・プロセスがFCN内で効率的に処理されます。

以下のコードを実行して、この動作を確認してください。ここでも、モデルのジョブ番号を取得してコード内で置き換える必要があります。

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import copy
from scipy.misc import imresize
import time

JOB_NUM = '20160725-164716-1f3e'  ## Remember to set this to be the job number for your model

MODEL_FILE = '/home/ubuntu/digits/digits/jobs/' + JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/home/ubuntu/digits/digits/jobs/' + JOB_NUM + '/snapshot_iter_270.caffemodel'    # Do not change

# Choose a random image to test against
RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = 'data/samples/w_' + RANDOM_IMAGE + '.jpg'                  

# Tell Caffe to use the GPU
caffe.set_mode_gpu()

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Initialize the Caffe model using the model trained in DIGITS
# This time the model input size is reshaped based on the randomly selected input image
net = caffe.Net(MODEL_FILE,PRETRAINED,caffe.TEST)
net.blobs['data'].reshape(1, 3, input_image.shape[0], input_image.shape[1])
net.reshape()
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

# This is just a colormap for displaying the results
my_cmap = copy.copy(plt.cm.get_cmap('jet')) # get a copy of the jet color map
my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values

# Feed the whole input image into the model for classification
start = time.time()
out = net.forward(data=np.asarray([transformer.preprocess('data', input_image)]))
end = time.time()

# Create an overlay visualization of the classification result
im = transformer.deprocess('data', net.blobs['data'].data[0])
classifications = out['softmax'][0]
classifications = imresize(classifications.argmax(axis=0),input_image.shape,interp='bilinear').astype('float')
classifications[classifications==0] = np.nan
plt.imshow(im)
plt.imshow(classifications,alpha=.5,cmap=my_cmap)
plt.show()

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

上記のコードを複数回実行すると、多くの場合、FCNの方がスライディング・ウィンドウによる物体検出方法より高い精度でクジラの顔の位置を特定できることが分かります。多くの場合、FCNはより多くのクジラを発見しますが、砕ける波や海面に反射する日光により混乱することもあります。ここでも、適切なデータ拡大を使用することにより、バックグラウンドの込み入った領域やクジラの胴体が原因で発生する誤検出を軽減することができます。

スライディング・ウィンドウ方式では10秒かかったのに対し、FCNの総推論時間は約1.5秒だという点に注目してください。つまり、何分の1かの時間でより良い品質の結果を得られたわけです。1回の効率的なパスで検出と分類を実行できる単一のモデルが得られるため、飛行機内などのリアルタイム・アプリケーションで検出器を配備する場合には、これが大きな利点になります。

トレーニング・データのデータ拡大以外にも、FCNによる物体検出方法の分類精度と位置特定精度を改善する一般的な方法がいくつかあります。この中でも最も一般的なのは、さまざまなスケールで複数回入力画像をネットワーク送信するというものです。
これにより、対象物体の出現におけるスケール変動へのモデルの許容度を改善できます。
より細かいまたはより粗い粒度の分類ヒートマップ出力を実現するために、ネットワーク層のストライドを修正することもできます。入力画像および分類ネットワークの複数のバージョンを同時に使用し、分類ヒートマップのマージ手順を実行することにより、最終的な分類および検出結果を大幅に改善できます。この方法のよく知られた例は、『OverFeat』という論文で紹介されています（図4）。

![overfeat](overfeat.png)
<h4 align="center">図4: 検出粒度向上の為のOverFeat手法</h4> 

### 演習:  

元に戻ってFCNテスト・コードを修正し、各種サイズの入力画像をネットワーク送信する効果を確認します。まず、テスト基準とする固定入力画像を選択します。次に、画像のサイズを手動で変更し、ネットワークに送信します。これを数回実行し、検出精度への効果を比較します。

## 物体検出方法4: DetectNet

画像内の各場所にある、最も可能性が高い物体を同時に分類し、回帰によりその物体に対応する境界ボックスを予想するようCNNをトレーニング(学習)する物体検出方法の最終クラスがあります。たとえば、以下のようなものです。

![yolo](yolo.png)

この方法には以下のような大きな利点があります。

* シンプルなワンショット検出、分類、および境界ボックス回帰パイプライン
* 非常に短いレイテンシ
* 強力で膨大なバックグラウンド・トレーニング・データにより実現される非常に低い誤検出率

この種のネットワークをトレーニング(学習)するには、すべての対象物体に正確な境界ボックスでラベルが付けられた、専用トレーニング・データが必要です。この種のトレーニング・データは非常に入手困難で、作成には多額のコストがかかります。ただし、物体検出問題にこの種のデータを利用できるのであれば、最適な方法であることはまず間違いありません。図5は、車両検出シナリオのラベル付きトレーニング・サンプルの例を示しています。

![kespry example](kespry_example.png)
<h4 align="center">図5: 3クラス物体検出シナリオの為のラベル付きデータ</h4> 

最近リリースされたDIGITS 4には、このクラスのモデルをトレーニング(学習)する機能が追加され、例としてDetectNetと呼ばれる新しい「標準ネットワーク」が提供されています。DetectNetを使用し、海のフルサイズの航空画像でRight Whale検出器のトレーニング(学習)を行います。

物体検出および境界ボックス回帰のために単一のCNNをトレーニング(学習)するにあたっての大きな課題は、複数の画像にさまざまな数の物体が存在する可能性があるという事実の処理にあります。場合によっては、物体をまったく含まない画像が存在することもあります。DetectNetは、多数の境界ボックス注釈を含む画像を、CNNでの推測を直接試みる固定次元データ表現に変換することにより、この問題を処理します。図6は、単一クラスの物体検出問題においてこの表現にデータがどのようにマッピングされるかを示しています。

![detectnet data rep](detectnet_data.png)
<h4 align="center">図6: DetectNetデータ表現</h4> 

前述のとおり、DetectNetは実際にはFCNですが、出力としてこのデータ表現を正確に生成するよう構成されています。DetectNetの層の大部分は、よく知られているGoogLeNetネットワークと同一です。図7は、トレーニング(学習)用のDetectNetアーキテクチャを示しています。

![detectnet training architecture](detectnet_training.png)
<h4 align="center">図7: DetectNetトレーニングアーキテクチャ</h4> 

このラボでは、DetectNetのトレーニング(学習)に適切なフォルダとフォーマットで、海のフルサイズの航空画像がすでに用意されています。まずは、このデータをDIGITSにインポートする必要があります。[Datasets]の[Images]ドロップダウンを使用し、[object detection]データセットを選択します。[New Object Detection Dataset]パネルが表示されたら、次の前処理オプションを設定します。

![OD data ingest](OD_ingest.png)

ではまず、このデータセットでDetectNetをトレーニング(学習)する手順を確認しましょう。このデータセットでDetectNetのトレーニング(学習)を完全に実行すると数時間かかるため、トレーニング済みモデルを実習用に用意しました。
メインのDIGITS画面に戻り、[Models]タブを使用します。[whale_detectnet]モデルを開き、このクローンを作成します。以下の変更を加えます。

* 新たに作成した[whales_detectnet]データセットを選択します
* トレーニング・エポック数を3に変更します
* バッチ・サイズを10に変更します

[Visualize]ボタンをクリックし、自由にネットワーク・アーキテクチャを視覚的に探索してください。

トレーニング(学習)を開始する準備ができたら、「whale_detectnet_2」のような新しい名前をモデルに付け、[create]をクリックします。このモデルを3エポック分トレーニング(学習)するだけでも約8分かかりますが、カバレッジと境界ボックスの両方のトレーニングおよび検証ロスの値がすでに減少していることが分かります。また、平均精度（mAP）スコアが上昇し始めていることも確認できます。mAPは、ネットワークによるクジラの顔の検出精度と、検証データセットに対するその境界ボックスの推定精度とを測る組み合わせ尺度です。

モデルのトレーニング(学習)が終了したら、事前にトレーニング済みの[whale_detectnet]モデルに戻ります。100トレーニング・エポック後、このモデルが低いトレーニングおよび検証ロス値だけではなく、高いmAPスコアにも収束することを確認できます。検証画像を基準にこのトレーニング済みモデルをテストし、クジラの顔を検出できるかどうかを確認しましょう。

手順は、視覚化方法を[Bounding boxes]に設定し、画像パス`/home/ubuntu/data/whale/data_336x224/val/images/000000118.png`を貼り付けるだけです。
[Show visualizations and statistics]チェックボックスを必ずオンにし、[Test One]をクリックします。DetectNetがクジラの顔の検出に成功し、以下のような境界ボックスを描画することを確認できます。

![detectnet success](detectnet_success.png)

`/home/ubuntu/data/whale/data_336x224/val/images/`フォルダの他の画像も自由にテストしてください。DetectNetが、隙間なく描画された境界ボックスでほとんどのクジラの顔を正確に検出でき、誤検出率が非常に低いことが分かります。さらに、DetectNetではきわめて高速に推論を実行できます。以下のセルを実行して、Caffeコマンド・ライン・インタフェースを使用し、DetectNetアーキテクチャを使用して推論ベンチマークを実行します。336x224ピクセルの単一画像をDetectNetに通すためにかかる平均時間は、たった22ミリ秒であることが分かります。

In [ ]:
!caffe time --model /home/ubuntu/data/whale/deploy.prototxt --gpu 0

## 質問の答え:

<a id='answer1'></a>
### 答え1

ランダム・パッチに結果的にクジラの顔が含まれることもありえます。顔は一般的に画像の非常に小さい一部にすぎず、クジラの顔をほぼまったく含まないランダム・バックグラウンド・パッチのサンプルを大量に使用しているため、この確率はそれほど高くありません。クジラの胴体と尾びれがバックグラウンド・セットに入る場合もありますが、クジラの顔の位置特定が対象のため、これは問題ありません。

質問1に戻るには、[ここ](#質問1)をクリックして下さい。

<a id='answer2'></a>
### 答え2

ランダムに選択された、顔ではないパッチをより多く使用し、ランダムな回転、フリップ、およびスケーリングを行って既存の顔パッチを増やすことにより、データセットのバランスを取ることをお勧めします。GoogleNetなど、より多くのトレーニング可能なパラメータを使ってモデルをトレーニング(学習)することもできるでしょう。

質問2に戻るには、[ここ](#質問2)をクリックして下さい。

<a id='answer3'></a>
### 答え3

複数のグリッド枠を一度にまとめてバッチ処理し、分類目的でバッチとしてネットワークに流すこともできます。これにより、並列処理をさらに活用でき、計算高速化の効果をGPUから得ることができます。

質問3に戻るには、[ここ](#質問3)をクリックして下さい。

<a id='answer4'></a>
### 答え4

層fc6～fc8を以下のものと置き換えます。

```layer {
  name: "fc6"
  type: "Convolution"
  bottom: "pool5"
  top: "fc6"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 256
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu6"
  type: "ReLU"
  bottom: "fc6"
  top: "fc6"
}
layer {
  name: "drop6"
  type: "Dropout"
  bottom: "fc6"
  top: "fc6"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "fc8"
  type: "Convolution"
  bottom: "fc6"
  top: "fc8"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}```

演習に戻るには[ここ](#exercise1)をクリックして下さい。

### 謝辞とデータの帰属

このラボでRight Whale画像を再利用する許可を下さったNOAA様に謝意を表します。
画像は以下のMMPA Research許可番号の下で収集いたしました。

MMPA 775-1600,      2005-2007

MMPA 775-1875,      2008-2013 

MMPA 17355,         2013-2018

写真提供: NOAA/NEFSC